### 回归

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName("TransformerExample")\
        .getOrCreate()

#### 1. 线性回归

In [5]:
from pyspark.ml.regression import LinearRegression

In [3]:
training = spark.read.format("libsvm")\
                     .load("../../data/sample_linear_regression_data.txt")

In [6]:
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [7]:
lrModel = lr.fit(training)

In [8]:
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

Coefficients: [0.0,0.32292516677405936,-0.3438548034562218,1.9156017023458414,0.05288058680386263,0.765962720459771,0.0,-0.15105392669186682,-0.21587930360904642,0.22025369188813426]
Intercept: 0.1598936844239736


In [9]:
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

numIterations: 6
objectiveHistory: [0.49999999999999994, 0.4967620357443381, 0.4936361664340463, 0.4936351537897608, 0.4936351214177871, 0.49363512062528014, 0.4936351206216114]
+--------------------+
|           residuals|
+--------------------+
|  -9.889232683103197|
|  0.5533794340053554|
|  -5.204019455758823|
| -20.566686715507508|
|    -9.4497405180564|
|  -6.909112502719486|
|  -10.00431602969873|
|   2.062397807050484|
|  3.1117508432954772|
| -15.893608229419382|
|  -5.036284254673026|
|   6.483215876994333|
|  12.429497299109002|
|  -20.32003219007654|
| -2.0049838218725005|
| -17.867901734183793|
|   7.646455887420495|
| -2.2653482182417406|
|-0.10308920436195645|
|  -1.380034070385301|
+--------------------+
only showing top 20 rows

RMSE: 10.189077
r2: 0.022861


#### 2. 决策树

In [10]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [11]:
data = spark.read.format("libsvm").load("../../data/sample_libsvm_data.txt")

In [12]:
featureIndexer = \
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4)\
    .fit(data)

In [13]:
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [17]:
dt = DecisionTreeRegressor(featuresCol="indexedFeatures")

In [18]:
pipeline = Pipeline(stages=[featureIndexer, dt])

In [19]:
model = pipeline.fit(trainingData)

In [20]:
predictions = model.transform(testData)

In [21]:
predictions.select("prediction", "label", "features").show(5)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|(692,[122,123,148...|
|       0.0|  0.0|(692,[124,125,126...|
|       0.0|  0.0|(692,[124,125,126...|
|       0.0|  0.0|(692,[124,125,126...|
|       0.0|  0.0|(692,[126,127,128...|
+----------+-----+--------------------+
only showing top 5 rows



In [22]:
evaluator = RegressionEvaluator(
        labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.254


In [23]:
treeModel = model.stages[1]
print(treeModel)

DecisionTreeRegressionModel: uid=DecisionTreeRegressor_1b51b87b00d7, depth=1, numNodes=3, numFeatures=692


#### 3. GBDT(Gradient Boosting Decision Tree)

In [24]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [26]:
data = spark.read.format("libsvm")\
                  .load("../../data/sample_libsvm_data.txt")

In [27]:
featureIndexer = \
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4)\
    .fit(data)

In [28]:
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [31]:
gbt = GBTRegressor(featuresCol="indexedFeatures", maxIter=10)

In [32]:
pipeline = Pipeline(stages=[featureIndexer, gbt])

In [33]:
model = pipeline.fit(trainingData)

In [34]:
predictions = model.transform(testData)

In [35]:
predictions.select("prediction", "label", "features").show(5)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|(692,[123,124,125...|
|       0.0|  0.0|(692,[124,125,126...|
|       0.0|  0.0|(692,[124,125,126...|
|       0.0|  0.0|(692,[126,127,128...|
|       0.0|  0.0|(692,[126,127,128...|
+----------+-----+--------------------+
only showing top 5 rows



In [36]:
evaluator = RegressionEvaluator(
        labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.188982


In [37]:
gbtModel = model.stages[1]
print(gbtModel)  # summary only

GBTRegressionModel: uid=GBTRegressor_2416d5cf9f5d, numTrees=10, numFeatures=692
